In [15]:
import pandas as pd
import numpy as np
import unicodedata
import os

df = pd.read_csv('../data/raw/2004-2021.tsv', sep='\t')

### Tratamento dos dados

In [16]:
df.head()
df.info() 

'''Observações do .info()
    - colunas de Datas estão no formato 'str', mas deveriam estar no formato 'datetime'
    - as colunas abaixo estão no formato 'str', mas deveriam estar no formato 'float': 
        11 MARGEM MÉDIA REVENDA
        13 PREÇO MÉDIO DISTRIBUIÇÃO       
        14 DESVIO PADRÃO DISTRIBUIÇÃO     
        15 PREÇO MÍNIMO DISTRIBUIÇÃO       
        16 PREÇO MÁXIMO DISTRIBUIÇÃO       
        17 COEF DE VARIAÇÃO DISTRIBUIÇÃO
'''

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120823 entries, 0 to 120822
Data columns (total 18 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   DATA INICIAL                   120823 non-null  object 
 1   DATA FINAL                     120823 non-null  object 
 2   REGIÃO                         120823 non-null  object 
 3   ESTADO                         120823 non-null  object 
 4   PRODUTO                        120823 non-null  object 
 5   NÚMERO DE POSTOS PESQUISADOS   120823 non-null  int64  
 6   UNIDADE DE MEDIDA              120823 non-null  object 
 7   PREÇO MÉDIO REVENDA            120823 non-null  float64
 8   DESVIO PADRÃO REVENDA          120823 non-null  float64
 9   PREÇO MÍNIMO REVENDA           120823 non-null  float64
 10  PREÇO MÁXIMO REVENDA           120823 non-null  float64
 11  MARGEM MÉDIA REVENDA           120823 non-null  object 
 12  COEF DE VARIAÇÃO REVENDA      

"Observações do .info()\n    - colunas de Datas estão no formato 'str', mas deveriam estar no formato 'datetime'\n    - as colunas abaixo estão no formato 'str', mas deveriam estar no formato 'float': \n        11 MARGEM MÉDIA REVENDA\n        13 PREÇO MÉDIO DISTRIBUIÇÃO       \n        14 DESVIO PADRÃO DISTRIBUIÇÃO     \n        15 PREÇO MÍNIMO DISTRIBUIÇÃO       \n        16 PREÇO MÁXIMO DISTRIBUIÇÃO       \n        17 COEF DE VARIAÇÃO DISTRIBUIÇÃO\n"

In [17]:
# Convertendo colunas de datas
df['DATA INICIAL'] = pd.to_datetime(df['DATA INICIAL'])
df['DATA FINAL'] = pd.to_datetime(df['DATA FINAL'])

# Convertendo colunas numéricas
colunas_float = [
    'MARGEM MÉDIA REVENDA',
    'PREÇO MÉDIO DISTRIBUIÇÃO',
    'DESVIO PADRÃO DISTRIBUIÇÃO',
    'PREÇO MÍNIMO DISTRIBUIÇÃO',
    'PREÇO MÁXIMO DISTRIBUIÇÃO',
    'COEF DE VARIAÇÃO DISTRIBUIÇÃO'
]

for coluna in colunas_float:
    df[coluna] = (
        df[coluna]
        .astype(str)           # garante que é string pra fazer replace
        .str.replace(',', '.', regex=False) # troca vírgula por ponto
        .replace('-', np.nan)  # trata o traço
        .astype(float)         # converte pra float
    )

    # Substituir NaN ou -99999 pela mediana válida
    mediana = df.loc[(df[coluna].notna()) & (df[coluna] != -99999), coluna].median()

    df[coluna] = df[coluna].replace(-99999, np.nan)  # marca -99999 como NaN
    df[coluna] = df[coluna].fillna(mediana)          # substitui NaN pela mediana

In [18]:
df['PRODUTO'].value_counts()

#OBS: Produtos com diferenciação devido a escrita

PRODUTO
GASOLINA COMUM        23570
GLP                   23561
ETANOL HIDRATADO      23440
ÓLEO DIESEL           21194
GNV                   14469
ÓLEO DIESEL S10        9113
OLEO DIESEL S10        2376
OLEO DIESEL            2351
GASOLINA ADITIVADA      749
Name: count, dtype: int64

In [19]:
# Função para remover acentos
def remover_acentos(texto):
    if isinstance(texto, str):
        return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    return texto

# Aplicando na coluna 'PRODUTO'
df['PRODUTO'] = df['PRODUTO'].apply(remover_acentos).str.upper().str.strip()

# Conferir novamente os valores únicos
print(df['PRODUTO'].value_counts())

PRODUTO
GASOLINA COMUM        23570
GLP                   23561
OLEO DIESEL           23545
ETANOL HIDRATADO      23440
GNV                   14469
OLEO DIESEL S10       11489
GASOLINA ADITIVADA      749
Name: count, dtype: int64


In [20]:
# Crie a pasta se ela não existir
os.makedirs('../data/processed', exist_ok=True)


# Salvar em CSV
df.to_csv('../data/processed/dados_tratados.csv', index=False)